In [1]:
import aif360
import numpy as np
import pandas as pd

from aif360.metrics import BinaryLabelDatasetMetric
from aif360.metrics import ClassificationMetric
from aif360.metrics.utils import compute_boolean_conditioning_vector

from aif360.metrics import utils
from aif360.datasets import BinaryLabelDataset
from aif360.datasets.multiclass_label_dataset import MulticlassLabelDataset
from aif360.algorithms.postprocessing import CalibratedEqOddsPostprocessing, EqOddsPostprocessing, RejectOptionClassification



import sklearn 
from sklearn.linear_model import LogisticRegression
import imblearn
import matplotlib.pyplot as plt 
from sklearn.metrics import classification_report, confusion_matrix
import math 


In [2]:
german_data = pd.read_csv('german_data.csv')


In [3]:
### set sensitive attribute equal to 'sex' or 'Age Group' ###
sen_att = 'sex'

In [4]:
#split data in a test val and train set

from sklearn.model_selection import train_test_split
X = german_data.loc[:, german_data.columns != 'approval']
y = german_data.loc[:, german_data.columns == 'approval']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify = y, random_state=0)

In [5]:
#apply SMOTE to the trainingset
from imblearn.over_sampling import SMOTE
os = SMOTE(random_state=4)

In [6]:
columns = X_train.columns
X_train_balanced, y_train_balanced=os.fit_sample(X_train, y_train)

In [7]:
balanced_german_train_df = pd.concat([X_train_balanced, y_train_balanced], axis=1, join="inner")

In [8]:
german_test_df = pd.concat([X_test, y_test], axis=1, join="inner")

In [9]:
from aif360.algorithms.preprocessing import DisparateImpactRemover
dataset_orig_train = aif360.datasets.BinaryLabelDataset(
    favorable_label=1,
    unfavorable_label=0,
    df=balanced_german_train_df,
    label_names=['approval'],
    protected_attribute_names=[sen_att])

In [10]:
from aif360.algorithms.preprocessing import DisparateImpactRemover
dataset_orig_test = aif360.datasets.BinaryLabelDataset(
    favorable_label=1,
    unfavorable_label=0,
    df=german_test_df,
    label_names=['approval'],
    protected_attribute_names=[sen_att])

In [11]:
from aif360.algorithms.preprocessing import DisparateImpactRemover
dataset_orig = aif360.datasets.BinaryLabelDataset(
    favorable_label=1,
    unfavorable_label=0,
    df=german_data,
    label_names=['approval'],
    protected_attribute_names=[sen_att])

In [12]:
privileged_groups = [{sen_att: 0}]
unprivileged_groups = [{sen_att: 1}]
cost_constraint = "fnr"
randseed = 12345679 

In [13]:
### logistic regression ### 

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_curve

# Placeholder for predicted and transformed datasets
dataset_orig_train_pred = dataset_orig_train.copy(deepcopy=True)
dataset_orig_test_pred = dataset_orig_test.copy(deepcopy=True)
dataset_new_test_pred = dataset_orig_test.copy(deepcopy=True)

# Logistic regression classifier and predictions for training data
scale_orig = StandardScaler()
X_train = scale_orig.fit_transform(dataset_orig_train.features)
y_train = dataset_orig_train.labels.ravel()
lmod = LogisticRegression()
lmod.fit(X_train, y_train)

fav_idx = np.where(lmod.classes_ == dataset_orig_train.favorable_label)[0][0]
y_train_pred_prob = lmod.predict_proba(X_train)[:,fav_idx]

X_test = scale_orig.transform(dataset_orig_test.features)
y_test_pred_prob = lmod.predict_proba(X_test)[:,fav_idx]

class_thresh = 0.5
dataset_orig_train_pred.scores = y_train_pred_prob.reshape(-1,1)

dataset_orig_test_pred.scores = y_test_pred_prob.reshape(-1,1)

y_train_pred = np.zeros_like(dataset_orig_train_pred.labels)
y_train_pred[y_train_pred_prob >= class_thresh] = dataset_orig_train_pred.favorable_label
y_train_pred[~(y_train_pred_prob >= class_thresh)] = dataset_orig_train_pred.unfavorable_label
dataset_orig_train_pred.labels = y_train_pred
    
y_test_pred = np.zeros_like(dataset_orig_test_pred.labels)
y_test_pred[y_test_pred_prob >= class_thresh] = dataset_orig_test_pred.favorable_label
y_test_pred[~(y_test_pred_prob >= class_thresh)] = dataset_orig_test_pred.unfavorable_label
dataset_orig_test_pred.labels = y_test_pred

In [14]:
# Metrics function
from collections import OrderedDict
from aif360.metrics import ClassificationMetric

def compute_metrics(dataset_true, dataset_pred, 
                    unprivileged_groups, privileged_groups,
                    disp = True):
    """ Compute the key metrics """
    classified_metric_pred = ClassificationMetric(dataset_true,
                                                 dataset_pred, 
                                                 unprivileged_groups=unprivileged_groups,
                                                 privileged_groups=privileged_groups)
    metrics = OrderedDict()
    metrics["Balanced accuracy"] = 0.5*(classified_metric_pred.true_positive_rate()+
                                             classified_metric_pred.true_negative_rate())
    metrics["Statistical parity difference"] = classified_metric_pred.statistical_parity_difference()
    metrics["Disparate impact"] = classified_metric_pred.disparate_impact()
    metrics["Average odds difference"] = classified_metric_pred.average_odds_difference()
    metrics["Equal opportunity difference"] = classified_metric_pred.equal_opportunity_difference()

    if disp:
        for k in metrics:
            print("%s = %.4f" % (k, metrics[k]))

In [15]:
### Reject Option Classification  Kamishima et al., 2012
ROC = RejectOptionClassification(privileged_groups = privileged_groups,
                             unprivileged_groups = unprivileged_groups)

ROC = ROC.fit(dataset_orig_test, dataset_orig_test_pred)
dataset_transf_test_pred = ROC.predict(dataset_orig_test_pred)


In [16]:
 metric_test_aft = compute_metrics(dataset_orig_test, dataset_transf_test_pred, 
                                      unprivileged_groups, privileged_groups,
                                      disp = True)

Balanced accuracy = 0.7440
Statistical parity difference = -0.0089
Disparate impact = 0.9832
Average odds difference = 0.0252
Equal opportunity difference = 0.1127


In [17]:
### extract test labels and prediction label for traditional model evaluation ###

y_test = dataset_orig_test.labels.ravel()
y_pred = dataset_transf_test_pred.labels.ravel()

### obtain performance metrics###

matrix = sklearn.metrics.confusion_matrix(y_test, y_pred)
accuracy_score = sklearn.metrics.accuracy_score(y_test, y_pred)
print(matrix, accuracy_score)
print(classification_report(y_test, y_pred))

[[49 11]
 [46 94]] 0.715
              precision    recall  f1-score   support

         0.0       0.52      0.82      0.63        60
         1.0       0.90      0.67      0.77       140

    accuracy                           0.71       200
   macro avg       0.71      0.74      0.70       200
weighted avg       0.78      0.71      0.73       200

